# Libraries

In [37]:
# using conda library 'ontogpt_fork'; naming irrelevant as this was previously going to be used to work on a different project.

# increase cell-width
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import pandas as pd
import re
import numpy as np
import os
import glob
from tqdm import tqdm # progress bar tracking

# API keys:
from dotenv import load_dotenv
load_dotenv()

pd.set_option('display.max_rows', None)
pd.set_option('display.width', 50)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 50)

import json
import jsonschema
from jsonschema import validate

# prompts for OpenAI

import openai_secret_manager
import openai
import time
import random

import tiktoken
from typing import List, Tuple

from collections import Counter

# Load SciERC dataset
* http://nlp.cs.washington.edu/sciIE/
* annotation guideline: http://nlp.cs.washington.edu/sciIE/annotation_guideline.pdf

* SciERC is an annotated dataset of 500 scientific abstracts (multi-sentence).
* Each article has 3 files:
    * .txt being the raw text.
    * .xml being the post-processed article.
    * .ann being the annotations.

## Annotation guideline

Relation link can not go beyond sentence boundary.

We define 4 asymmetric relation types (Used-for, Feature-of, Hyponym-of, Part-of), together with 2 symmetric relation types (Compare, Conjunction).
B always points to A for asymmetric relations.

* Used-for: B is used for A, B models A, A is trained on B, B exploits A, A is based on B. E.g.
    * The TISPER system has been designed to enable many text applications.
    * Our method models user proficiency.
    * Our algorithms exploits local soothness.

* Feature-of: B belongs to A, B is a feature of A, B is under A domain. E.g.
    * prior knowledge of the model
    * genre-specific regularities of discourse structure
    * English text in science domain

* Hyponym-of: B is a hyponym of A, B is a type of A. E.g.
    * TUIT is a software library
    * NLP applications such as machine translation and language generation

* Part-of: B is a part of A... E.g.
    * The system includes two models: speech recognition and natural language understanding
    * We incorporate NLU module to the system.

* Compare: Symmetric relation (use blue to denote entity). Opposite of conjunction, compare two models/methods, or listing two opposing entities. E.g.
    * Unlike the quantitative prior, the qualitative prior is often ignored...

* Conjunction: Symmetric relation (use blue to denote entity). Function as similar role or use/incorporate
with. E.g.
    * obtained from human expert or knowledge base
    * NLP applications such as machine translation and language generation

Coreference:
* Anaphora and Cataphora:
    * We introduce a machine reading system... The system...
    * The prior knowledge include...Such knowledge can be applied to...

* Coreferring noun phrase:
    * We develop a part-of-speech tagging system...The POS tagger...

## Explore .ann files

In [2]:
# Directory path:
directory_path = './benchmarking/input/sciERC_raw'

# Get the list of .ann files in the directory
ann_files = glob.glob(os.path.join(directory_path, '*.ann'))

# Iterate over each .ann file
for ann_file in ann_files:
    # Load the .ann file
    with open(ann_file, 'r') as f:
        annotations = f.readlines()

    # Print the .ann file name
    print("Annotations in", ann_file)

    # Print the annotations
    for annotation in annotations:
        print(annotation.strip())

    print("-----------")

Annotations in ./benchmarking/input/sciERC_raw\A00-1024.ann
T1	Generic 26 32	system
T2	Task 37 63	categorizing unknown words
T3	Generic 71 77	system
T4	Method 94 122	multi-component architecture
T5	Generic 136 145	component
T6	OtherScientificTerm 192 205	unknown words
T7	Generic 240 250	components
T8	OtherScientificTerm 267 272	names
T9	OtherScientificTerm 279 294	spelling errors
T10	Generic 303 312	component
T11	Method 322 348	decision tree architecture
T12	OtherScientificTerm 401 413	unknown word
T13	Generic 421 427	system
T14	Material 459 479	live closed captions
T15	OtherScientificTerm 523 536	unknown words
R1	USED-FOR Arg1:T4 Arg2:T3
R2	USED-FOR Arg1:T7 Arg2:T8
R3	USED-FOR Arg1:T11 Arg2:T10
R4	PART-OF Arg1:T5 Arg2:T4
R5	USED-FOR Arg1:T7 Arg2:T9
R6	HYPONYM-OF Arg1:T7 Arg2:T5
R7	COREF Arg1:T13 Arg2:T3
R8	USED-FOR Arg1:T5 Arg2:T6
R9	EVALUATE-FOR Arg1:T14 Arg2:T13
R10	CONJUNCTION Arg1:T8 Arg2:T9
R11	COREF Arg1:T10 Arg2:T7
R12	USED-FOR Arg1:T1 Arg2:T2
R13	COREF Arg1:T1 Arg2:T3
--------

Annotations in ./benchmarking/input/sciERC_raw\C88-2166.ann
T1	Method 19 42	natural language system
T2	OtherScientificTerm 53 74	computational lexicon
T3	Generic 83 89	system
T4	Method 344 351	COMPLEX
T5	OtherScientificTerm 358 379	computational lexicon
T6	OtherScientificTerm 413 439	shared lexical information
T7	Task 453 494	Natural Language Processing (NLP) systems
T8	Material 565 602	machine-readable dictionaries (MRD's)
R1	USED-FOR Arg1:T2 Arg2:T1
R2	COREF Arg1:T3 Arg2:T1
R3	HYPONYM-OF Arg1:T4 Arg2:T5
R4	USED-FOR Arg1:T4 Arg2:T7
R5	COREF Arg1:T5 Arg2:T2
-----------
Annotations in ./benchmarking/input/sciERC_raw\C90-1002.ann
T1	Method 4 24	deterministic parser
T2	Method 94 115	deterministic parsers
T3	Generic 125 127	it
T4	Method 143 180	symbolic and connectionist components
T5	Method 187 210	connectionist component
T6	OtherScientificTerm 235 243	patterns
T7	Method 263 297	rules  of a  deterministic grammar
T8	Method 341 360	hybrid architecture
T9	Method 377 383	parser
T10	Method 41

Annotations in ./benchmarking/input/sciERC_raw\CVPR_2009_11_abs.ann
T1	OtherScientificTerm 23 50	pan-tilt-zoom (PTZ) cameras
T2	OtherScientificTerm 77 91	panoramic area
T3	OtherScientificTerm 105 128	high resolution imagery
T4	Task 144 174	automated surveillance systems
T5	OtherScientificTerm 189 200	PTZ cameras
T6	Generic 251 261	algorithms
T7	OtherScientificTerm 274 331	prior knowledge of intrinsic parameters of the PTZ camera
T8	OtherScientificTerm 345 365	relative positioning
T9	OtherScientificTerm 370 381	orientation
T10	OtherScientificTerm 397 408	PTZ cameras
T11	Method 458 475	mapping algorithm
T12	OtherScientificTerm 493 513	relative positioning
T13	OtherScientificTerm 518 529	orientation
T14	OtherScientificTerm 542 553	PTZ cameras
T15	Method 565 589	unified polynomial model
T16	OtherScientificTerm 663 673	PTZ camera
T17	OtherScientificTerm 678 696	relative positions
T18	Generic 749 758	algorithm
T19	Metric 790 814	computational complexity
T20	Metric 828 839	flexibility
T21	Met

Annotations in ./benchmarking/input/sciERC_raw\E93-1004.ann
T1	Method 31 48	modal language LT
T2	OtherScientificTerm 63 85	constraints  on  trees
T3	Method 95 113	extension  LT (LF)
T4	OtherScientificTerm 129 185	constraints  on  trees decorated with feature structures
T5	Generic 226 235	languages
T6	Method 274 296	grammatical frameworks
T7	Method 381 385	GPSG
T8	Method 407 414	LT (LF)
T9	Method 443 458	modal languages
T10	Method 504 525	constraint formalisms
R1	HYPONYM-OF Arg1:T1 Arg2:T5
R2	USED-FOR Arg1:T5 Arg2:T6
R3	COREF Arg1:T9 Arg2:T5
R4	USED-FOR Arg1:T7 Arg2:T8
R5	USED-FOR Arg1:T1 Arg2:T2
R6	USED-FOR Arg1:T3 Arg2:T4
R7	HYPONYM-OF Arg1:T3 Arg2:T5
R8	COREF Arg1:T8 Arg2:T3
-----------
Annotations in ./benchmarking/input/sciERC_raw\E93-1023.ann
T1	OtherScientificTerm 105 114	ambiguity
T2	Task 122 132	generation
T3	OtherScientificTerm 233 242	ambiguity
T4	Method 250 276	MORphological PArser MORPA
T5	Method 298 339	probabilistic context-free grammar (PCFG)
T6	Generic 347 349	it
T7	Met

Annotations in ./benchmarking/input/sciERC_raw\H91-1077.ann
T1	Generic 3 9	method
T2	Task 14 30	sense resolution
T3	Material 62 69	WordNet
T4	Material 75 100	on-line  lexical database
T5	OtherScientificTerm 121 139	semantic relations
T6	OtherScientificTerm 143 151	synonymy
T7	OtherScientificTerm 155 163	antonymy
T8	OtherScientificTerm 167 175	hyponymy
T9	OtherScientificTerm 179 187	meronymy
T10	OtherScientificTerm 191 223	causal and troponymic entailment
T11	Material 277 284	WordNet
T12	OtherScientificTerm 319 345	semantically related words
T13	Task 382 398	sense resolution
T14	Task 408 423	text processing
T15	Generic 502 512	procedures
T16	OtherScientificTerm 580 598	alternative senses
T17	OtherScientificTerm 608 623	polysemous word
T18	OtherScientificTerm 739 754	polysemous word
T19	OtherScientificTerm 978 993	polysemous word
T20	Material 1106 1113	WordNet
T21	OtherScientificTerm 1150 1167	semantic distance
T22	Generic 1372 1381	procedure
T23	Task 1432 1453	information retrieval
T24	

Annotations in ./benchmarking/input/sciERC_raw\ICCV_2015_393_abs.ann
T1	Task 37 59	loss-aware predictions
T2	Task 37 90	loss-aware predictions in image segmentation settings
T3	Task 63 90	image segmentation settings
T4	OtherScientificTerm 101 120	evaluation function
T5	Metric 128 165	Intersection-over-Union (IoU) measure
T6	Method 200 226	image segmentation systems
T7	Generic 253 272	dominant approaches
T8	Generic 278 283	first
T9	OtherScientificTerm 301 326	Expected-IoU (EIoU) score
T10	OtherScientificTerm 330 379	Expected-Intersection-over-Expected-Union (EIoEU)
T11	Generic 389 404	second approach
T12	OtherScientificTerm 425 429	EIoU
T13	Task 559 584	image seg-mentation tasks
T14	Generic 689 696	methods
T15	OtherScientificTerm 705 710	EIoEU
T16	Method 705 724	EIoEU approximation
T17	Generic 807 817	approaches
T18	Task 855 879	image segmentation tasks
R1	EVALUATE-FOR Arg1:T5 Arg2:T6
R2	HYPONYM-OF Arg1:T5 Arg2:T4
R3	HYPONYM-OF Arg1:T8 Arg2:T7
R4	HYPONYM-OF Arg1:T11 Arg2:T7
R5	COREF Arg

Annotations in ./benchmarking/input/sciERC_raw\INTERSPEECH_2014_20_abs.ann
T1	Task 26 65	multilingual feature-level data sharing
T2	Method 70 123	Deep Neural Network (DNN) stacked bottleneck features
T3	Task 177 200	language identification
T4	Material 286 308	multilingual resources
T5	Material 331 352	IARPA-Babel languages
T6	OtherScientificTerm 363 382	bottleneck features
T7	Generic 447 452	those
T8	Generic 532 536	data
T9	Task 582 603	multilingual training
R1	USED-FOR Arg1:T2 Arg2:T1
R2	USED-FOR Arg1:T3 Arg2:T4
R3	COMPARE Arg1:T7 Arg2:T6
R4	USED-FOR Arg1:T8 Arg2:T9
-----------
Annotations in ./benchmarking/input/sciERC_raw\INTERSPEECH_2014_28_abs.ann
T1	Method 28 80	statistical singing voice conversion (SVC) technique
T2	Method 86 114	direct waveform modification
T3	OtherScientificTerm 128 149	spectrum differential
T4	OtherScientificTerm 167 179	voice timbre
T5	Method 244 251	vocoder
T6	OtherScientificTerm 264 297	converted singing voice waveforms
T7	Method 299 302	SVC
T8	OtherScient

Annotations in ./benchmarking/input/sciERC_raw\NIPS_2001_21_abs.ann
T1	Task 25 48	blind source separation
T2	OtherScientificTerm 63 92	instantaneous linear mixtures
T3	OtherScientificTerm 104 117	mixing matrix
T4	OtherScientificTerm 178 197	sparsity of sources
T5	OtherScientificTerm 249 266	signal dictionary
T6	Metric 294 315	quality of separation
T7	OtherScientificTerm 353 375	multi scale transforms
T8	OtherScientificTerm 385 411	wavelet or wavelet packets
T9	OtherScientificTerm 447 461	local features
T10	OtherScientificTerm 486 494	sparsity
T11	OtherScientificTerm 575 583	features
T12	Generic 631 640	algorithm
T13	Material 656 681	noise-free and noisy data
T14	Material 700 717	simulated signals
T15	Material 719 733	musical sounds
T16	Material 738 744	images
T17	Metric 784 802	separation quality
R1	USED-FOR Arg1:T2 Arg2:T1
R2	USED-FOR Arg1:T5 Arg2:T4
R3	EVALUATE-FOR Arg1:T6 Arg2:T4
R4	EVALUATE-FOR Arg1:T13 Arg2:T12
R5	CONJUNCTION Arg1:T14 Arg2:T15
R6	CONJUNCTION Arg1:T15 Arg2:T16
R7	H

Annotations in ./benchmarking/input/sciERC_raw\P05-1067.ann
T1	Task 2 51	Syntax-based statistical machine translation (MT)
T2	Method 71 89	statistical models
T3	Material 95 110	structured data
T4	Task 143 194	syntax-based statistical machine translation system
T5	Method 208 262	probabilistic synchronous dependency insertion grammar
T6	Method 267 308	Synchronous dependency insertion grammars
T7	Method 328 348	synchronous grammars
T8	OtherScientificTerm 362 378	dependency trees
T9	Generic 405 413	approach
T10	Method 434 441	grammar
T11	Material 449 465	parallel corpora
T12	Method 494 509	graphical model
T13	Task 520 544	machine translation task
T14	Method 579 613	stochastic tree-to-tree transducer
T15	Method 633 667	polynomial time decoding algorithm
T16	Generic 678 683	model
T17	Method 719 728	MT system
T18	Metric 741 787	NIST and Bleu automatic MT evaluation software
T19	Generic 817 823	system
T20	Generic 841 856	baseline system
T21	Method 872 882	IBM models
T22	Metric 893 922	translat

Annotations in ./benchmarking/input/sciERC_raw\W05-1308.ann
T1	Method 30 63	fully automated extraction system
T2	Method 73 78	IntEx
T3	Task 94 123	gene and protein interactions
T4	Material 129 144	biomedical text
T5	Generic 151 159	approach
T6	OtherScientificTerm 253 268	syntactic roles
T7	OtherScientificTerm 286 305	biological entities
T8	OtherScientificTerm 325 361	biomedical and linguistic ontologies
T9	OtherScientificTerm 447 462	syntactic roles
T10	Method 520 537	extraction system
T11	OtherScientificTerm 581 613	multiple and nested interactions
T12	Method 700 718	extraction systems
T13	Method 739 751	IntEx system
T14	OtherScientificTerm 812 843	pattern engineering requirement
R1	COMPARE Arg1:T13 Arg2:T12
R2	HYPONYM-OF Arg1:T2 Arg2:T1
R3	USED-FOR Arg1:T1 Arg2:T3
R4	USED-FOR Arg1:T4 Arg2:T3
R5	COREF Arg1:T5 Arg2:T1
R6	USED-FOR Arg1:T8 Arg2:T7
R7	COREF Arg1:T10 Arg2:T5
R8	USED-FOR Arg1:T10 Arg2:T11
R9	COREF Arg1:T13 Arg2:T10
-----------
Annotations in ./benchmarking/input/sciERC_raw\

## Extract entities and relationships data into separate cols, corresponding to the .ann file

In [3]:
# Specify the directory path where your files are stored
directory_path = './benchmarking/input/sciERC_raw'

# Initialize empty lists to store the data
abstracts = []
entities = []
relationships = []

# Get the list of .ann files in the directory
ann_files = glob.glob(os.path.join(directory_path, '*.ann'))

# Iterate over each .ann file
for ann_file in ann_files:
    # Load the .ann file
    with open(ann_file, 'r') as f:
        lines = f.readlines()

    # Extract the abstract (title of .ann file)
    abstract = os.path.splitext(os.path.basename(ann_file))[0]

    # Store the annotations in a dictionary
    ann_dict = {}
    rel_dict = {}
    for line in lines:
        line_parts = line.strip().split('\t')
        if line_parts[0].startswith('T'):
            ann_id = line_parts[0]
            ann_type = line_parts[1].split()[0]
            start_offset = int(line_parts[1].split()[1])
            end_offset = int(line_parts[1].split()[2])
            annotated_text = line_parts[2]
            ann_dict[ann_id] = {
                'Annotation ID': ann_id,
                'Entity': ann_type,
                'Start Offset': start_offset,
                'End Offset': end_offset,
                'Annotated Text': annotated_text
            }
        elif line_parts[0].startswith('R'):
            rel_id = line_parts[0]
            rel_type = line_parts[1].split()[0]
            arg1_id = line_parts[1].split()[1].split(':')[1]
            arg2_id = line_parts[1].split()[2].split(':')[1]
            rel_dict[rel_id] = {
                'Relationship ID': rel_id,
                'Type': rel_type,
                'Arg1': arg1_id,
                'Arg2': arg2_id
            }

    # Append the data to the lists
    abstracts.append(abstract)
    entities.append(ann_dict)
    relationships.append(rel_dict)

# Create a DataFrame from the collected data
data = {
    'abstract_id': abstracts,
    'entities': entities,
    'relationships': relationships
}

annotations_df = pd.DataFrame(data)

In [4]:
print(len(annotations_df))
annotations_df.head()

# export to excel to view

500


,abstract_id,entities,relationships
0,A00-1024,"{'T1': {'Annotation ID': 'T1', 'Entity': 'Gene...","{'R1': {'Relationship ID': 'R1', 'Type': 'USED..."
1,A00-2023,"{'T1': {'Annotation ID': 'T1', 'Entity': 'Gene...","{'R1': {'Relationship ID': 'R1', 'Type': 'COMP..."
2,A88-1001,"{'T1': {'Annotation ID': 'T1', 'Entity': 'Meth...","{'R1': {'Relationship ID': 'R1', 'Type': 'FEAT..."
3,A88-1003,"{'T1': {'Annotation ID': 'T1', 'Entity': 'Meth...","{'R1': {'Relationship ID': 'R1', 'Type': 'PART..."
4,A92-1010,"{'T1': {'Annotation ID': 'T1', 'Entity': 'Task...","{'R1': {'Relationship ID': 'R1', 'Type': 'USED..."


### Create populated_rels column, replacing entity IDs in the relationships col.

In [5]:
# Create a new column 'populated_rels' in annotations_df
annotations_df['populated_rels'] = None

# Iterate over each row in annotations_df
for index, row in annotations_df.iterrows():
    # Get the entities and relationships dictionaries for the current row
    entities_dict = row['entities']
    relationships_dict = row['relationships']
    
    # Create a new dictionary to store the populated relationships
    populated_rels_dict = {}
    
    # Iterate over each relationship in relationships_dict
    for rel_id, rel_info in relationships_dict.items():
        # Get the argument IDs
        arg1_id = rel_info['Arg1']
        arg2_id = rel_info['Arg2']
        
        # Get the corresponding annotated texts from entities_dict
        arg1_text = entities_dict[arg1_id]['Annotated Text']
        arg2_text = entities_dict[arg2_id]['Annotated Text']
        
        # Create the populated relationship string
        rel_text = f"{arg1_text} {rel_info['Type']} {arg2_text}"
        
        # Store the populated relationship in the new dictionary
        populated_rels_dict[rel_id] = {
            'Relationship ID': rel_id,
            'Rel': rel_text
        }
    
    # Assign the populated relationships dictionary to the 'populated_rels' column
    annotations_df.at[index, 'populated_rels'] = populated_rels_dict


In [6]:
print(len(annotations_df))
annotations_df.head()

# export to excel to view

500


,abstract_id,entities,relationships,populated_rels
0,A00-1024,"{'T1': {'Annotation ID': 'T1', 'Entity': 'Gene...","{'R1': {'Relationship ID': 'R1', 'Type': 'USED...","{'R1': {'Relationship ID': 'R1', 'Rel': 'multi..."
1,A00-2023,"{'T1': {'Annotation ID': 'T1', 'Entity': 'Gene...","{'R1': {'Relationship ID': 'R1', 'Type': 'COMP...","{'R1': {'Relationship ID': 'R1', 'Rel': 'ranki..."
2,A88-1001,"{'T1': {'Annotation ID': 'T1', 'Entity': 'Meth...","{'R1': {'Relationship ID': 'R1', 'Type': 'FEAT...","{'R1': {'Relationship ID': 'R1', 'Rel': 'feedb..."
3,A88-1003,"{'T1': {'Annotation ID': 'T1', 'Entity': 'Meth...","{'R1': {'Relationship ID': 'R1', 'Type': 'PART...","{'R1': {'Relationship ID': 'R1', 'Rel': 'prono..."
4,A92-1010,"{'T1': {'Annotation ID': 'T1', 'Entity': 'Task...","{'R1': {'Relationship ID': 'R1', 'Type': 'USED...","{'R1': {'Relationship ID': 'R1', 'Rel': 'displ..."


### Create a simplified populated_rels col, and a count_of_rels col

In [7]:
# Create a new column 'simplified_populated_rels' in annotations_df
annotations_df['simplified_populated_rels'] = None

# Iterate over each row in annotations_df
for index, row in annotations_df.iterrows():
    # Get the populated relationships dictionary for the current row
    populated_rels_dict = row['populated_rels']
    
    # Extract the relationship strings from populated_rels_dict
    rel_strings = [rel_info['Rel'] for rel_info in populated_rels_dict.values()]
    
    # Assign the simplified relationship strings to the 'simplified_populated_rels' column
    annotations_df.at[index, 'simplified_populated_rels'] = rel_strings
    
# Create a new column 'rel_count' in annotations_df
annotations_df['rel_count'] = annotations_df['simplified_populated_rels'].apply(lambda x: len(x))


In [8]:
print(len(annotations_df))
annotations_df.head()

500


,abstract_id,entities,relationships,populated_rels,simplified_populated_rels,rel_count
0,A00-1024,"{'T1': {'Annotation ID': 'T1', 'Entity': 'Gene...","{'R1': {'Relationship ID': 'R1', 'Type': 'USED...","{'R1': {'Relationship ID': 'R1', 'Rel': 'multi...","[multi-component architecture USED-FOR system,...",13
1,A00-2023,"{'T1': {'Annotation ID': 'T1', 'Entity': 'Gene...","{'R1': {'Relationship ID': 'R1', 'Type': 'COMP...","{'R1': {'Relationship ID': 'R1', 'Rel': 'ranki...",[ranking algorithm COMPARE lattice-based appro...,9
2,A88-1001,"{'T1': {'Annotation ID': 'T1', 'Entity': 'Meth...","{'R1': {'Relationship ID': 'R1', 'Type': 'FEAT...","{'R1': {'Relationship ID': 'R1', 'Rel': 'feedb...","[feedback FEATURE-OF discourse, videodisc imag...",9
3,A88-1003,"{'T1': {'Annotation ID': 'T1', 'Entity': 'Meth...","{'R1': {'Relationship ID': 'R1', 'Type': 'PART...","{'R1': {'Relationship ID': 'R1', 'Rel': 'prono...",[pronominal anaphora resolution module PART-OF...,3
4,A92-1010,"{'T1': {'Annotation ID': 'T1', 'Entity': 'Task...","{'R1': {'Relationship ID': 'R1', 'Type': 'USED...","{'R1': {'Relationship ID': 'R1', 'Rel': 'displ...",[display of graphical information USED-FOR cog...,3


## Load .txt files (same name as corresponding .ann files)

In [9]:
# Path to the directory containing the .txt files
txt_directory = './benchmarking/input/sciERC_raw'

# Create an empty DataFrame to store the text data
text_df = pd.DataFrame(columns=['abstract_id', 'text'])

# Iterate over the rows of annotations_df
for index, row in annotations_df.iterrows():
    # Extract the abstract name
    abstract = row['abstract_id']
    
    # Construct the path to the corresponding .txt file
    txt_file_path = os.path.join(txt_directory, f'{abstract}.txt')
    
    # Load the text from the .txt file
    with open(txt_file_path, 'r') as f:
        text = f.read()
    
    # Append the abstract name and text to text_df
    text_df = pd.concat([text_df, pd.DataFrame({'abstract_id': [abstract], 'text': [text]})], ignore_index=True)
    
print(len(text_df))
text_df.head()


500


,abstract_id,text
0,A00-1024,\nThis paper introduces a system for categori...
1,A00-2023,\nThis paper presents a new approach to stati...
2,A88-1001,\n This paper describes a domain independent ...
3,A88-1003,"\n In this paper, we describe the pronominal..."
4,A92-1010,\nIn our current research into the design of ...


In [10]:
# Perform an inner join between annotations_df and text_df based on the 'Abstract' column
scierc_full = pd.merge(text_df, annotations_df, on='abstract_id', how='inner')

print(len(scierc_full))
scierc_full.head()

500


,abstract_id,text,entities,relationships,populated_rels,simplified_populated_rels,rel_count
0,A00-1024,\nThis paper introduces a system for categori...,"{'T1': {'Annotation ID': 'T1', 'Entity': 'Gene...","{'R1': {'Relationship ID': 'R1', 'Type': 'USED...","{'R1': {'Relationship ID': 'R1', 'Rel': 'multi...","[multi-component architecture USED-FOR system,...",13
1,A00-2023,\nThis paper presents a new approach to stati...,"{'T1': {'Annotation ID': 'T1', 'Entity': 'Gene...","{'R1': {'Relationship ID': 'R1', 'Type': 'COMP...","{'R1': {'Relationship ID': 'R1', 'Rel': 'ranki...",[ranking algorithm COMPARE lattice-based appro...,9
2,A88-1001,\n This paper describes a domain independent ...,"{'T1': {'Annotation ID': 'T1', 'Entity': 'Meth...","{'R1': {'Relationship ID': 'R1', 'Type': 'FEAT...","{'R1': {'Relationship ID': 'R1', 'Rel': 'feedb...","[feedback FEATURE-OF discourse, videodisc imag...",9
3,A88-1003,"\n In this paper, we describe the pronominal...","{'T1': {'Annotation ID': 'T1', 'Entity': 'Meth...","{'R1': {'Relationship ID': 'R1', 'Type': 'PART...","{'R1': {'Relationship ID': 'R1', 'Rel': 'prono...",[pronominal anaphora resolution module PART-OF...,3
4,A92-1010,\nIn our current research into the design of ...,"{'T1': {'Annotation ID': 'T1', 'Entity': 'Task...","{'R1': {'Relationship ID': 'R1', 'Type': 'USED...","{'R1': {'Relationship ID': 'R1', 'Rel': 'displ...",[display of graphical information USED-FOR cog...,3


In [11]:
scierc_full['word_count'] = scierc_full['text'].apply(lambda x: len(str(x).split()))

smallest_word_count = scierc_full['word_count'].min()
largest_word_count = scierc_full['word_count'].max()

print(smallest_word_count)
print(largest_word_count)

23
316


### Distinct relationship types

* Seems as if EVALUATE-FOR is not mentioned in the annotation_guideline.pdf

In [12]:
# Extract unique 'Types' from the 'relationships' column
unique_types = scierc_full['relationships'].apply(lambda x: [rel_info['Type'] for rel_info in x.values()]).explode().unique().tolist()

# Print the list of unique 'Types' prettily
for type in unique_types:
    print(type)


USED-FOR
PART-OF
HYPONYM-OF
COREF
EVALUATE-FOR
CONJUNCTION
COMPARE
FEATURE-OF
nan


# Prompt generation to detect rels
* Attempting to use similar prompt format as used in main KG_construction, for consistency.
* Aim is to benchmark against a rel-annotated dataset (scierc)

In [13]:
print(len(scierc_full))
scierc_full.head()

500


,abstract_id,text,entities,relationships,populated_rels,simplified_populated_rels,rel_count,word_count
0,A00-1024,\nThis paper introduces a system for categori...,"{'T1': {'Annotation ID': 'T1', 'Entity': 'Gene...","{'R1': {'Relationship ID': 'R1', 'Type': 'USED...","{'R1': {'Relationship ID': 'R1', 'Rel': 'multi...","[multi-component architecture USED-FOR system,...",13,86
1,A00-2023,\nThis paper presents a new approach to stati...,"{'T1': {'Annotation ID': 'T1', 'Entity': 'Gene...","{'R1': {'Relationship ID': 'R1', 'Type': 'COMP...","{'R1': {'Relationship ID': 'R1', 'Rel': 'ranki...",[ranking algorithm COMPARE lattice-based appro...,9,85
2,A88-1001,\n This paper describes a domain independent ...,"{'T1': {'Annotation ID': 'T1', 'Entity': 'Meth...","{'R1': {'Relationship ID': 'R1', 'Type': 'FEAT...","{'R1': {'Relationship ID': 'R1', 'Rel': 'feedb...","[feedback FEATURE-OF discourse, videodisc imag...",9,58
3,A88-1003,"\n In this paper, we describe the pronominal...","{'T1': {'Annotation ID': 'T1', 'Entity': 'Meth...","{'R1': {'Relationship ID': 'R1', 'Type': 'PART...","{'R1': {'Relationship ID': 'R1', 'Rel': 'prono...",[pronominal anaphora resolution module PART-OF...,3,77
4,A92-1010,\nIn our current research into the design of ...,"{'T1': {'Annotation ID': 'T1', 'Entity': 'Task...","{'R1': {'Relationship ID': 'R1', 'Type': 'USED...","{'R1': {'Relationship ID': 'R1', 'Rel': 'displ...",[display of graphical information USED-FOR cog...,3,84


#### Previous prompt used in main

In [13]:
# todo

In [23]:
# changed article to sentence.
# simplified.

# rel types:
# USED-FOR
# FEATURE-OF
# HYPONYM-OF
# PART-OF
# EVALUATE-FOR
# COMPARE
# CONJUNCTION
# COREF

# Create a function to generate the prompt based on the row values
def generate_prompt(row):

    input_text = row['text']
    
    prompt = f'''Give me a comma-separated list of detected triples from the provided abstract, as long as the entities and relationships are in this list of restricted entity and relationship types:
    ###
    The entity types must be in: [Task, Method, Evaluation Metric, Material, Other Scientific Terms, Generic].
    ###
    Here are some examples of each type of entity that may be detected:
    Task: Applications, problems to solve, systems to construct. E.g. information extraction, machine reading system, image segmentation.
    Method: Methods , models, systems to use, or tools, components of a system, frameworks. E.g. language model, CORENLP, POS parser, kernel method.
    Evaluation Metric: Metrics, measures, or entities that can express quality of a system/method. E.g. F1, BLEU, Precision, Recall, ROC curve, mean reciprocal rank, mean-squared error, robustness,
    time complexity.
    Material: Data, datasets, resources, Corpus, Knowledge base. E.g. image data, speech data, stereo images, bilingual dictionary, paraphrased questions, CoNLL,
    Panntreebank, WordNet, Wikipedia.
    Other Scientific Terms: Phrases that are a scientific terms but do not fall into any of the above classes E.g. physical or geometric constraints, qualitative prior knowledge, discourse structure, syntactic rule,
    discourse structure, tree, node, tree kernel, features, noise, criteria.
    Generic: General terms or pronouns that may refer to a entity but are not themselves informative, often used as connection words. E.g model, approach, prior knowledge, them, it.
    ###
    The relationships types must be in: [USED-FOR, FEATURE-OF, HYPONYM-OF, PART-OF, EVALUATE-FOR, COMPARE, CONJUNCTION, COREF].
    ###
    Here are some examples of triples that may be detected, that pertain to the restricted entities and relationships (note that you do not need to give me the type of entity, just the entity itself):
    USED-FOR: "The TISPER system has been designed to enable many text applications, Our method models user proficiency" would become "TISPER system USED-FOR text applications".
    FEATURE-OF: "prior knowledge of the model" would become "prior knowledge FEATURE-OF model".
    HYPONYM-OF: "TUIT is a software library." would become "TUIT HYPONYM-OF software library".
    PART-OF: "We incorporate NLU module to the system." would become "NLU module PART-OF system".
    EVALUATE-FOR: "our work improves both the quality and the efficiency of entity summarization" would become "efficiency EVALUATE-FOR entity summarization".
    COMPARE: "Unlike the quantitative prior, the qualitative prior is often ignored" would become "quantitative prior COMPARE qualitative prior".
    CONJUNCTION: "obtained from human expert or knowledge base" would become "human expert CONJUNCTION knowledge base".
    COREF: "We introduce a machine reading system... The system..." would become "machine reading system COREF system".
    ###
    Desired response format: ###
    [triple1, triple2, triple3]
    ###
    Example response: ###
    ["machine reading system COREF system", "prior knowledge FEATURE-OF model", "TISPER system USED-FOR text applications"]
    ###
    The provided abstract is: ###
    {input_text}
    ###
    '''
    
    return prompt

# Add a new 'prompt' column by applying the generate_prompt function to each row
scierc_full['prompt'] = scierc_full.apply(generate_prompt, axis=1)
scierc_full['prompt'] = scierc_full['prompt'].astype(str)


In [24]:
print(len(scierc_full))
scierc_full.head()

500


,abstract_id,text,entities,relationships,populated_rels,simplified_populated_rels,rel_count,word_count,prompt
0,A00-1024,\nThis paper introduces a system for categorizing unknown words . The system is based on a multi-component architecture where each component is responsible for identifying one class of unknown words . The focus of this paper is the components that identify names and spelling errors . Each component uses a decision tree architecture to combine multiple types of evidence about the unknown word . The system is evaluated using data from live closed captions - a genre replete with a wide variety of unknown words .\n,"{'T1': {'Annotation ID': 'T1', 'Entity': 'Generic', 'Start Offset': 26, 'End Offset': 32, 'Annotated Text': 'system'}, 'T2': {'Annotation ID': 'T2', 'Entity': 'Task', 'Start Offset': 37, 'End Offset': 63, 'Annotated Text': 'categorizing unknown words'}, 'T3': {'Annotation ID': 'T3', 'Entity': 'Generic', 'Start Offset': 71, 'End Offset': 77, 'Annotated Text': 'system'}, 'T4': {'Annotation ID': 'T4', 'Entity': 'Method', 'Start Offset': 94, 'End Offset': 122, 'Annotated Text': 'multi-component architecture'}, 'T5': {'Annotation ID': 'T5', 'Entity': 'Generic', 'Start Offset': 136, 'End Offset': 145, 'Annotated Text': 'component'}, 'T6': {'Annotation ID': 'T6', 'Entity': 'OtherScientificTerm', 'Start Offset': 192, 'End Offset': 205, 'Annotated Text': 'unknown words'}, 'T7': {'Annotation ID': 'T7', 'Entity': 'Generic', 'Start Offset': 240, 'End Offset': 250, 'Annotated Text': 'components'}, 'T8': {'Annotation ID': 'T8', 'Entity': 'OtherScientificTerm', 'Start Offset': 267, 'End Offset': 272, 'Annotated Text': 'names'}, 'T9': {'Annotation ID': 'T9', 'Entity': 'OtherScientificTerm', 'Start Offset': 279, 'End Offset': 294, 'Annotated Text': 'spelling errors'}, 'T10': {'Annotation ID': 'T10', 'Entity': 'Generic', 'Start Offset': 303, 'End Offset': 312, 'Annotated Text': 'component'}, 'T11': {'Annotation ID': 'T11', 'Entity': 'Method', 'Start Offset': 322, 'End Offset': 348, 'Annotated Text': 'decision tree architecture'}, 'T12': {'Annotation ID': 'T12', 'Entity': 'OtherScientificTerm', 'Start Offset': 401, 'End Offset': 413, 'Annotated Text': 'unknown word'}, 'T13': {'Annotation ID': 'T13', 'Entity': 'Generic', 'Start Offset': 421, 'End Offset': 427, 'Annotated Text': 'system'}, 'T14': {'Annotation ID': 'T14', 'Entity': 'Material', 'Start Offset': 459, 'End Offset': 479, 'Annotated Text': 'live closed captions'}, 'T15': {'Annotation ID': 'T15', 'Entity': 'OtherScientificTerm', 'Start Offset': 523, 'End Offset': 536, 'Annotated Text': 'unknown words'}}","{'R1': {'Relationship ID': 'R1', 'Type': 'USED-FOR', 'Arg1': 'T4', 'Arg2': 'T3'}, 'R2': {'Relationship ID': 'R2', 'Type': 'USED-FOR', 'Arg1': 'T7', 'Arg2': 'T8'}, 'R3': {'Relationship ID': 'R3', 'Type': 'USED-FOR', 'Arg1': 'T11', 'Arg2': 'T10'}, 'R4': {'Relationship ID': 'R4', 'Type': 'PART-OF', 'Arg1': 'T5', 'Arg2': 'T4'}, 'R5': {'Relationship ID': 'R5', 'Type': 'USED-FOR', 'Arg1': 'T7', 'Arg2': 'T9'}, 'R6': {'Relationship ID': 'R6', 'Type': 'HYPONYM-OF', 'Arg1': 'T7', 'Arg2': 'T5'}, 'R7': {'Relationship ID': 'R7', 'Type': 'COREF', 'Arg1': 'T13', 'Arg2': 'T3'}, 'R8': {'Relationship ID': 'R8', 'Type': 'USED-FOR', 'Arg1': 'T5', 'Arg2': 'T6'}, 'R9': {'Relationship ID': 'R9', 'Type': 'EVALUATE-FOR', 'Arg1': 'T14', 'Arg2': 'T13'}, 'R10': {'Relationship ID': 'R10', 'Type': 'CONJUNCTION', 'Arg1': 'T8', 'Arg2': 'T9'}, 'R11': {'Relationship ID': 'R11', 'Type': 'COREF', 'Arg1': 'T10', 'Arg2': 'T7'}, 'R12': {'Relationship ID': 'R12', 'Type': 'USED-FOR', 'Arg1': 'T1', 'Arg2': 'T2'}, 'R13': {'Relationship ID': 'R13', 'Type': 'COREF', 'Arg1': 'T1', 'Arg2': 'T3'}}","{'R1': {'Relationship ID': 'R1', 'Rel': 'multi-component architecture USED-FOR system'}, 'R2': {'Relationship ID': 'R2', 'Rel': 'components USED-FOR names'}, 'R3': {'Relationship ID': 'R3', 'Rel': 'decision tree architecture USED-FOR component'}, 'R4': {'Relationship ID': 'R4', 'Rel': 'component PART-OF multi-com

# Exploring prompts

In [25]:
# Set the display options to show all text and split by newline
pd.set_option('display.max_colwidth', None)
pd.set_option('display.expand_frame_repr', False)

# Iterate over each row in the 'prompt' column
for prompt in scierc_full['prompt'][:1]:
    print(prompt)
    print()

Give me a comma-separated list of detected triples from the provided abstract, as long as the entities and relationships are in this list of restricted entity and relationship types:
    ###
    The entity types must be in: [Task, Method, Evaluation Metric, Material, Other Scientific Terms, Generic].
    ###
    Here are some examples of each type of entity that may be detected:
    Task: Applications, problems to solve, systems to construct. E.g. information extraction, machine reading system, image segmentation.
    Method: Methods , models, systems to use, or tools, components of a system, frameworks. E.g. language model, CORENLP, POS parser, kernel method.
    Evaluation Metric: Metrics, measures, or entities that can express quality of a system/method. E.g. F1, BLEU, Precision, Recall, ROC curve, mean reciprocal rank, mean-squared error, robustness,
    time complexity.
    Material: Data, datasets, resources, Corpus, Knowledge base. E.g. image data, speech data, stereo images, bi

In [26]:
# convert to list to explore
all_prompts = scierc_full['prompt'].tolist()
print(len(all_prompts))

500


## Investigate average length of tokens

* Important for pricing forecast and LLM API restrictions

### with tiktoken

In [27]:
encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")
#encoding = tiktoken.get_encoding("cl100k_base")

In [28]:
# fns to count strings in list of prompts:

def num_tokens_from_string(string: str, encoding_name: str) -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

def count_tokens_in_list(prompt_list: list, encoding_name: str) -> list:
    """Returns a list of integers representing the number of tokens in each string in the input list."""
    token_counts = []
    for prompt in prompt_list:
        num_tokens = num_tokens_from_string(prompt, encoding_name)
        token_counts.append(num_tokens)
    return token_counts


In [29]:
encoding_name = "cl100k_base" # used for gpt-3.5-turbo
token_counts = count_tokens_in_list(all_prompts, encoding_name)

In [30]:
min_tokens = min(token_counts)
max_tokens = max(token_counts)
total_tokens = sum(i for i in token_counts if isinstance(i, int))
average_tokens = total_tokens / len(all_prompts)

print(f"Number of tokens on the smallest prompt: {min_tokens}")
print(f"Number of tokens on the largest prompt: {max_tokens}")
print(f"Total number of tokens for all prompts: {total_tokens}")
print(f"Average number of tokens in all_prompts: {average_tokens}")

Number of tokens on the smallest prompt: 758
Number of tokens on the largest prompt: 1178
Total number of tokens for all prompts: 443434
Average number of tokens in all_prompts: 886.868


In [31]:
# Estimating max response tokens if prompt works correctly (update to reflect prompt used)

#num_tokens_from_string("facing risk: yes. type of risk: thisis some text for a risk type.", "cl100k_base")

num_tokens_from_string(
    
    '''
["NL evaluations COMPARE Message Understanding Conferences", "evaluation methodology USED-FOR mature, practical applications", "methodology USED-FOR comparative evaluation of SLS systems", "evaluation methodology EVALUATE-FOR competing claims and identifying promising technical approaches", "methodology USED-FOR automatic evaluation of question-answering NL systems", "methodology can be used with speech or text input", "methodology FEATURE-OF heart", "DARPA SLS program PART-OF methodology implementation", "NL interfaces HYPONYM-OF speech understanding", "SLS systems HYPONYM-OF question-answering NL systems", "SLS systems PART-OF DARPA Spoken Language Systems program", "SLS systems COMPARE NL evaluations other than Message Understanding Conferences", "researchers COREF they"]
    '''
                       ,"cl100k_base" )

169

### Truncate tokens of long prompts

**** NO NEED TO TRUNCATE PROMPTS WORKING WITH scierc_full ****
**** All are below token input threshold ****

* gpt-3.5-turbo has max tokens of 4,096 tokens
* This includes prompt and response tokens combined.
* response tokens should be short due to the attempt at prompt restrictions;
    * i.e. Provide answers only in the format of <facing risk: <'yes'/'no'>. type of risk: < risk type >.> and nothing else.
* so a generous estimate of response tokens would be 100, providing gpt-3.5-turbo successfully adheres to above prompting.
* Therefore truncate prompt tokens to 3500 to be safe.

# Feed prompts into LLM

* Initially was working from a list of prompts.
* Modified to work from df and append response directly to df in new 'responses' col.
    * Allows for linking to sent_id without feeding id into prompt.

In [32]:
# Create new df to contain responses at corresponding rows.
# MODIFY TO SPECIFY AMOUNT OF INPUT.

scierc_full_responses = scierc_full.copy()
scierc_full_responses.head(1)

,abstract_id,text,entities,relationships,populated_rels,simplified_populated_rels,rel_count,word_count,prompt
0,A00-1024,\nThis paper introduces a system for categorizing unknown words . The system is based on a multi-component architecture where each component is responsible for identifying one class of unknown words . The focus of this paper is the components that identify names and spelling errors . Each component uses a decision tree architecture to combine multiple types of evidence about the unknown word . The system is evaluated using data from live closed captions - a genre replete with a wide variety of unknown words .\n,"{'T1': {'Annotation ID': 'T1', 'Entity': 'Generic', 'Start Offset': 26, 'End Offset': 32, 'Annotated Text': 'system'}, 'T2': {'Annotation ID': 'T2', 'Entity': 'Task', 'Start Offset': 37, 'End Offset': 63, 'Annotated Text': 'categorizing unknown words'}, 'T3': {'Annotation ID': 'T3', 'Entity': 'Generic', 'Start Offset': 71, 'End Offset': 77, 'Annotated Text': 'system'}, 'T4': {'Annotation ID': 'T4', 'Entity': 'Method', 'Start Offset': 94, 'End Offset': 122, 'Annotated Text': 'multi-component architecture'}, 'T5': {'Annotation ID': 'T5', 'Entity': 'Generic', 'Start Offset': 136, 'End Offset': 145, 'Annotated Text': 'component'}, 'T6': {'Annotation ID': 'T6', 'Entity': 'OtherScientificTerm', 'Start Offset': 192, 'End Offset': 205, 'Annotated Text': 'unknown words'}, 'T7': {'Annotation ID': 'T7', 'Entity': 'Generic', 'Start Offset': 240, 'End Offset': 250, 'Annotated Text': 'components'}, 'T8': {'Annotation ID': 'T8', 'Entity': 'OtherScientificTerm', 'Start Offset': 267, 'End Offset': 272, 'Annotated Text': 'names'}, 'T9': {'Annotation ID': 'T9', 'Entity': 'OtherScientificTerm', 'Start Offset': 279, 'End Offset': 294, 'Annotated Text': 'spelling errors'}, 'T10': {'Annotation ID': 'T10', 'Entity': 'Generic', 'Start Offset': 303, 'End Offset': 312, 'Annotated Text': 'component'}, 'T11': {'Annotation ID': 'T11', 'Entity': 'Method', 'Start Offset': 322, 'End Offset': 348, 'Annotated Text': 'decision tree architecture'}, 'T12': {'Annotation ID': 'T12', 'Entity': 'OtherScientificTerm', 'Start Offset': 401, 'End Offset': 413, 'Annotated Text': 'unknown word'}, 'T13': {'Annotation ID': 'T13', 'Entity': 'Generic', 'Start Offset': 421, 'End Offset': 427, 'Annotated Text': 'system'}, 'T14': {'Annotation ID': 'T14', 'Entity': 'Material', 'Start Offset': 459, 'End Offset': 479, 'Annotated Text': 'live closed captions'}, 'T15': {'Annotation ID': 'T15', 'Entity': 'OtherScientificTerm', 'Start Offset': 523, 'End Offset': 536, 'Annotated Text': 'unknown words'}}","{'R1': {'Relationship ID': 'R1', 'Type': 'USED-FOR', 'Arg1': 'T4', 'Arg2': 'T3'}, 'R2': {'Relationship ID': 'R2', 'Type': 'USED-FOR', 'Arg1': 'T7', 'Arg2': 'T8'}, 'R3': {'Relationship ID': 'R3', 'Type': 'USED-FOR', 'Arg1': 'T11', 'Arg2': 'T10'}, 'R4': {'Relationship ID': 'R4', 'Type': 'PART-OF', 'Arg1': 'T5', 'Arg2': 'T4'}, 'R5': {'Relationship ID': 'R5', 'Type': 'USED-FOR', 'Arg1': 'T7', 'Arg2': 'T9'}, 'R6': {'Relationship ID': 'R6', 'Type': 'HYPONYM-OF', 'Arg1': 'T7', 'Arg2': 'T5'}, 'R7': {'Relationship ID': 'R7', 'Type': 'COREF', 'Arg1': 'T13', 'Arg2': 'T3'}, 'R8': {'Relationship ID': 'R8', 'Type': 'USED-FOR', 'Arg1': 'T5', 'Arg2': 'T6'}, 'R9': {'Relationship ID': 'R9', 'Type': 'EVALUATE-FOR', 'Arg1': 'T14', 'Arg2': 'T13'}, 'R10': {'Relationship ID': 'R10', 'Type': 'CONJUNCTION', 'Arg1': 'T8', 'Arg2': 'T9'}, 'R11': {'Relationship ID': 'R11', 'Type': 'COREF', 'Arg1': 'T10', 'Arg2': 'T7'}, 'R12': {'Relationship ID': 'R12', 'Type': 'USED-FOR', 'Arg1': 'T1', 'Arg2': 'T2'}, 'R13': {'Relationship ID': 'R13', 'Type': 'COREF', 'Arg1': 'T1', 'Arg2': 'T3'}}","{'R1': {'Relationship ID': 'R1', 'Rel': 'multi-component architecture USED-FOR system'}, 'R2': {'Relationship ID': 'R2', 'Rel': 'components USED-FOR names'}, 'R3': {'Relationship ID': 'R3', 'Rel': 'decision tree architecture USED-FOR component'}, 'R4': {'Relationship ID': 'R4', 'Rel': 'component PART-OF multi-com

In [38]:
openai.api_key = os.getenv('OPENAI_KEY')

## Prompt engineering

In [40]:
# model:
GPT_MODEL = "gpt-3.5-turbo"

# Retry parameters
max_retries = 3
retry_delay = 5  # in seconds

# Initialize a new column 'responses' in the dataframe
scierc_full_responses.loc[:, 'responses'] = ''

def generate_responses(input_df):
    for idx, row in tqdm(input_df.iterrows(), total=len(input_df), desc="Generating responses"):
        try:
            query = row['prompt']

            if query is None or query.strip() == '':
                print(f"Empty prompt at index {idx}. Skipping this row.")
                continue

            retries = 0
            while retries < max_retries:
                try:
                    response = openai.ChatCompletion.create(
                        model=GPT_MODEL,
                        messages=[
                            {"role": "system", "content": "You will perform triple detection, adhering to the restricted types of entities and relations listed in the query."},
                            {"role": "user", "content": query}, # In general, gpt-3.5-turbo-0301 does not pay strong attention to the system message, and therefore important instructions are often better placed in a user message.
                        ],
                        temperature=0,
                        max_tokens=1000,  # max tokens in response. token limit (4096) must be < largest query + message content + max_tokens
                    )

                    response_content = response['choices'][0]['message']['content']
                    print(response_content)  # Print just the message content

                    # Add the response to the 'responses' column of the dataframe
                    input_df.loc[idx, 'responses'] = response_content

                    break  # Break out of the retry loop if successful

                except Exception as e:
                    print(f"Error generating response for prompt at index {idx}: {str(e)}")
                    print(f"Query: {query}")
                    retries += 1
                    print(f"Retrying in {retry_delay} seconds...")
                    time.sleep(retry_delay)

            if retries == max_retries:
                print(f"Max retries reached for prompt at index {idx}. Skipping this row.")

            # Add a delay between requests to avoid overwhelming the API
            time.sleep(random.uniform(0.5, 1.5))

        except Exception as e:
            print(f"Error generating response for prompt at index {idx}: {str(e)}")
            print(f"Query: {query}")

## Run prompts

In [41]:
generate_responses(scierc_full_responses)

Generating responses:   0%|          | 0/500 [00:00<?, ?it/s]

["system categorizing unknown words USED-FOR identifying names and spelling errors", "multi-component architecture PART-OF system", "component responsible for identifying one class of unknown words FEATURE-OF decision tree architecture"]


Generating responses:   0%|          | 1/500 [00:04<34:49,  4.19s/it]

["approach USED-FOR statistical sentence generation", "alternative phrases FEATURE-OF packed sets", "packed sets FEATURE-OF trees", "packed sets FEATURE-OF forests", "representation OFFERS-ADVANTAGES-IN compactness", "representation OFFERS-ADVANTAGES-IN ability to represent syntactic information", "ranking algorithm FEATURE-OF efficient ranking", "previous approach COMPARE lattice-based approach", "experimental results EVALUATE-FOR improvements over simple enumeration or a lattice-based approach"]


Generating responses:   0%|          | 2/500 [00:10<45:32,  5.49s/it]

...

Generating responses: 100%|█████████▉| 498/500 [50:18<00:12,  6.07s/it]

["proper nouns RECOGNITION-OF morphological analysis PROBLEM-OF Japanese text processing", "Japanese information extraction FRAMEWORK-OF recent years", "Multi-lingual Evaluation Task (MET) CONSIDER Japanese morphological analysis problem", "morphological analyzer AMORPH RECOGNIZES NE items", "NE items RECOGNITION-OF dictionary lookup stage", "NE items IDENTIFICATION-OF segmented strings RULE APPLICATION stage"]


Generating responses: 100%|█████████▉| 499/500 [50:24<00:05,  5.90s/it]

["automatic summarization USED-FOR Internet services", "MUC USED-FOR next generation Internet", "SUMMAC USED-FOR next generation Internet", "automatic summarization PROPOSES models", "models FEATURE-OF summary generation", "tasks INITIATED-BY SUMMAC-1", "positive feature vectors USED-FOR generic summaries", "negative feature vectors USED-FOR generic summaries", "text model USED-FOR adhoc task", "relationship BETWEEN nouns AND verbs USED-FOR text model", "discourse segment PART-OF text model", "sentences RANKED-BY text model", "user-directed summaries GENERATED-BY text model", "NormF OF best summary EQUALS 0.456 FOR adhoc task", "NormF OF fixed summary EQUALS 0.447 FOR adhoc task", "NormF OF best summary EQUALS 0.4090 FOR categorization task", "NormF OF fixed summary EQUALS 0.4023 FOR categorization task", "system OUTPERFORMS average system IN categorization task"]


Generating responses: 100%|██████████| 500/500 [50:34<00:00,  6.07s/it]


In [42]:
scierc_full_responses.head()

,abstract_id,text,entities,relationships,populated_rels,simplified_populated_rels,rel_count,word_count,prompt,responses
0,A00-1024,\nThis paper introduces a system for categori...,"{'T1': {'Annotation ID': 'T1', 'Entity': 'Gene...","{'R1': {'Relationship ID': 'R1', 'Type': 'USED...","{'R1': {'Relationship ID': 'R1', 'Rel': 'multi...","[multi-component architecture USED-FOR system,...",13,86,Give me a comma-separated list of detected tri...,"[""system categorizing unknown words USED-FOR i..."
1,A00-2023,\nThis paper presents a new approach to stati...,"{'T1': {'Annotation ID': 'T1', 'Entity': 'Gene...","{'R1': {'Relationship ID': 'R1', 'Type': 'COMP...","{'R1': {'Relationship ID': 'R1', 'Rel': 'ranki...",[ranking algorithm COMPARE lattice-based appro...,9,85,Give me a comma-separated list of detected tri...,"[""approach USED-FOR statistical sentence gener..."
2,A88-1001,\n This paper describes a domain independent ...,"{'T1': {'Annotation ID': 'T1', 'Entity': 'Meth...","{'R1': {'Relationship ID': 'R1', 'Type': 'FEAT...","{'R1': {'Relationship ID': 'R1', 'Rel': 'feedb...","[feedback FEATURE-OF discourse, videodisc imag...",9,58,Give me a comma-separated list of detected tri...,"[""natural language interface USED-FOR database..."
3,A88-1003,"\n In this paper, we describe the pronominal...","{'T1': {'Annotation ID': 'T1', 'Entity': 'Meth...","{'R1': {'Relationship ID': 'R1', 'Type': 'PART...","{'R1': {'Relationship ID': 'R1', 'Rel': 'prono...",[pronominal anaphora resolution module PART-OF...,3,77,Give me a comma-separated list of detected tri...,"[""pronominal anaphora resolution module PART-O..."
4,A92-1010,\nIn our current research into the design of ...,"{'T1': {'Annotation ID': 'T1', 'Entity': 'Task...","{'R1': {'Relationship ID': 'R1', 'Type': 'USED...","{'R1': {'Relationship ID': 'R1', 'Rel': 'displ...",[display of graphical information USED-FOR cog...,3,84,Give me a comma-separated list of detected tri...,"[""design cognitively well-motivated interfaces..."


## Compare count of rels found to annotated count.
* Very unlikely to get a hard match of the entire list of rels between simplified_populated_rels and responses.
* Also unlikely to get items to match across lists.

* Can analyse count of rels combined with manual evaluation.

* seems to have formed new rel outside of restricted rel list: OFFERS-ADVANTAGES-IN

In [43]:
scierc_full_responses['responses_rels_detected'] = scierc_full_responses['responses'].apply(lambda x: len(x.split(',')))
scierc_full_responses.head(5)

,abstract_id,text,entities,relationships,populated_rels,simplified_populated_rels,rel_count,word_count,prompt,responses,responses_rels_detected
0,A00-1024,\nThis paper introduces a system for categori...,"{'T1': {'Annotation ID': 'T1', 'Entity': 'Gene...","{'R1': {'Relationship ID': 'R1', 'Type': 'USED...","{'R1': {'Relationship ID': 'R1', 'Rel': 'multi...","[multi-component architecture USED-FOR system,...",13,86,Give me a comma-separated list of detected tri...,"[""system categorizing unknown words USED-FOR i...",3
1,A00-2023,\nThis paper presents a new approach to stati...,"{'T1': {'Annotation ID': 'T1', 'Entity': 'Gene...","{'R1': {'Relationship ID': 'R1', 'Type': 'COMP...","{'R1': {'Relationship ID': 'R1', 'Rel': 'ranki...",[ranking algorithm COMPARE lattice-based appro...,9,85,Give me a comma-separated list of detected tri...,"[""approach USED-FOR statistical sentence gener...",9
2,A88-1001,\n This paper describes a domain independent ...,"{'T1': {'Annotation ID': 'T1', 'Entity': 'Meth...","{'R1': {'Relationship ID': 'R1', 'Type': 'FEAT...","{'R1': {'Relationship ID': 'R1', 'Rel': 'feedb...","[feedback FEATURE-OF discourse, videodisc imag...",9,58,Give me a comma-separated list of detected tri...,"[""natural language interface USED-FOR database...",3
3,A88-1003,"\n In this paper, we describe the pronominal...","{'T1': {'Annotation ID': 'T1', 'Entity': 'Meth...","{'R1': {'Relationship ID': 'R1', 'Type': 'PART...","{'R1': {'Relationship ID': 'R1', 'Rel': 'prono...",[pronominal anaphora resolution module PART-OF...,3,77,Give me a comma-separated list of detected tri...,"[""pronominal anaphora resolution module PART-O...",7
4,A92-1010,\nIn our current research into the design of ...,"{'T1': {'Annotation ID': 'T1', 'Entity': 'Task...","{'R1': {'Relationship ID': 'R1', 'Type': 'USED...","{'R1': {'Relationship ID': 'R1', 'Rel': 'displ...",[display of graphical information USED-FOR cog...,3,84,Give me a comma-separated list of detected tri...,"[""design cognitively well-motivated interfaces...",3


### Quantitative comparisons

In [44]:
rel_count_sum = scierc_full_responses['rel_count'].sum()
responses_rels_detected_sum = scierc_full_responses['responses_rels_detected'].sum()

print("Sum of rel_count:", rel_count_sum)
print("Sum of responses_rels_detected:", responses_rels_detected_sum)


Sum of rel_count: 6394
Sum of responses_rels_detected: 3242


In [45]:
# Define the target texts
target_texts = ['USED-FOR', 'FEATURE-OF', 'HYPONYM-OF', 'PART-OF', 'EVALUATE-FOR', 'COMPARE', 'CONJUNCTION', 'COREF']

# Initialize the dictionary
rel_summary = {text: 0 for text in target_texts}

# Count the occurrences of each target text
for row in scierc_full_responses['responses']:
    phrases = row.split(', ')
    for text in target_texts:
        for phrase in phrases:
            if text in phrase:
                rel_summary[text] += 1

# Print the summary statistics
print(rel_summary)

# Print the sum of values
print("Sum of Values:", sum(rel_summary.values))

{'USED-FOR': 619, 'FEATURE-OF': 613, 'HYPONYM-OF': 77, 'PART-OF': 413, 'EVALUATE-FOR': 380, 'COMPARE': 162, 'CONJUNCTION': 3, 'COREF': 10}


TypeError: 'builtin_function_or_method' object is not iterable

In [ ]:
# Above implies that there are some rels generated outside of the restricted options;
# Or, may be slight mis-matches with letter-casing, lack of hyphen, etc.

# Final export for manual verification

In [46]:
# # export to excel for manual verification:
scierc_full_responses.to_excel("./benchmarking/results/scierc_rel_detection_with_ent_type_responses.xlsx")

# # export to pickle in case of future use.
scierc_full_responses.to_pickle("./benchmarking/results/scierc_rel_detection_with_ent_type_responses.pkl")

In [47]:
reimport_scierc_results = scierc_full_responses.copy()

### remove rows where abstract ID had missing annotation file (C88-2132), or no rels detected (other 2)

In [48]:

# List of abstract values to be excluded
excluded_abstracts = ['IJCAI_2013_4_abs', 'AAAI_2008_262_abs', 'C88-2132']

# Filter out rows with excluded abstract values
reimport_scierc_results = reimport_scierc_results[~reimport_scierc_results['abstract_id'].isin(excluded_abstracts)]
print(len(reimport_scierc_results))
reimport_scierc_results.head(1)

497


,abstract_id,text,entities,relationships,populated_rels,simplified_populated_rels,rel_count,word_count,prompt,responses,responses_rels_detected
0,A00-1024,\nThis paper introduces a system for categori...,"{'T1': {'Annotation ID': 'T1', 'Entity': 'Gene...","{'R1': {'Relationship ID': 'R1', 'Type': 'USED...","{'R1': {'Relationship ID': 'R1', 'Rel': 'multi...","[multi-component architecture USED-FOR system,...",13,86,Give me a comma-separated list of detected tri...,"[""system categorizing unknown words USED-FOR i...",3


## Add comparison cols:
1) exact rel in annotated but not in responses.
2) corresponding count of rels.
3) exact rel in responses but not in annotated.
4) corresponding count of rels. 
5) exact rel in responses and annotations.
6) corresponding count of rels.

In [49]:
# Remove punctuation contained in most responses
reimport_scierc_results['responses'] = reimport_scierc_results['responses'].str.replace('[\[\]"]', '', regex=True)

# Trim leading and trailing spaces/newlines
# reimport_scierc_results['responses'] = reimport_scierc_results['responses'].apply(lambda x: [item.strip() for item in x])

# Display the updated DataFrame
reimport_scierc_results.head(1)


,abstract_id,text,entities,relationships,populated_rels,simplified_populated_rels,rel_count,word_count,prompt,responses,responses_rels_detected
0,A00-1024,\nThis paper introduces a system for categori...,"{'T1': {'Annotation ID': 'T1', 'Entity': 'Gene...","{'R1': {'Relationship ID': 'R1', 'Type': 'USED...","{'R1': {'Relationship ID': 'R1', 'Rel': 'multi...","[multi-component architecture USED-FOR system,...",13,86,Give me a comma-separated list of detected tri...,system categorizing unknown words USED-FOR ide...,3


In [50]:
# determine if comparison cols are list of items or string.
# Assuming the column name is 'col_name'
col_names = ('simplified_populated_rels', 'responses')

for col in col_names:
    col_name = col
    # Check the type of values in the column
    value_type = type(reimport_scierc_results.loc[0, col_name])

    # Check if the column contains a list
    if value_type == list:
        print(f"The '{col_name}' column contains a list of items.")
    else:
        print(f"The '{col_name}' column contains strings.")

# output must be same format for accurate comparison.

TypeError: 'float' object is not callable

In [104]:
# Assuming the column name is 'responses'
col_name = 'responses'

# Convert the string to a list of items
reimport_scierc_results[col_name] = reimport_scierc_results[col_name].str.split(',')

# Check the updated type of values in the column
value_type = type(reimport_scierc_results.loc[0, col_name])

# Check if the column contains a list
if value_type == list:
    print(f"The '{col_name}' column has been converted to a list of items.")
else:
    print(f"Error: The '{col_name}' column could not be converted to a list.")
reimport_scierc_results.head(1)


The 'responses' column has been converted to a list of items.


In [105]:
# strip
reimport_scierc_results['responses'] = reimport_scierc_results['responses'].apply(lambda x: [item.strip() for item in x])
reimport_scierc_results.head(1)

In [107]:
# Step 2: Create 'rel_annotated_but_not_detected' column
reimport_scierc_results['rel_annotated_but_not_detected'] = reimport_scierc_results.apply(lambda row: [rel for rel in row['simplified_populated_rels'] if rel not in row['responses']], axis=1)
reimport_scierc_results['rel_annotated_but_not_detected_count'] = reimport_scierc_results['rel_annotated_but_not_detected'].apply(len)

# Step 3: Create 'rel_detected_but_not_annotated' column
reimport_scierc_results['rel_detected_but_not_annotated'] = reimport_scierc_results.apply(lambda row: [rel for rel in row['responses'] if rel not in row['simplified_populated_rels']], axis=1)
reimport_scierc_results['rel_detected_but_not_annotated_count'] = reimport_scierc_results['rel_detected_but_not_annotated'].apply(len)

# Step 4: Create 'annotated_rel_detected' column
reimport_scierc_results['annotated_rel_detected'] = reimport_scierc_results.apply(lambda row: [rel for rel in row['responses'] if rel in row['simplified_populated_rels']], axis=1)
reimport_scierc_results['annotated_rel_detected_count'] = reimport_scierc_results['annotated_rel_detected'].apply(len)


In [109]:
reimport_scierc_results.head()

,abstract_id,text,entities,relationships,populated_rels,simplified_populated_rels,rel_count,prompt,responses,responses_rels_detected,rel_annotated_but_not_detected,rel_annotated_but_not_detected_count,rel_detected_but_not_annotated,rel_detected_but_not_annotated_count,annotated_rel_detected,annotated_rel_detected_count
0,A00-1024,\nThis paper introduces a system for categori...,"{'T1': {'Annotation ID': 'T1', 'Entity': 'Gene...","{'R1': {'Relationship ID': 'R1', 'Type': 'USED...","{'R1': {'Relationship ID': 'R1', 'Rel': 'multi...","[multi-component architecture USED-FOR system,...",13,Give me a comma-separated list of relationship...,"[system PART-OF multi-component architecture, ...",5,"[multi-component architecture USED-FOR system,...",13,"[system PART-OF multi-component architecture, ...",5,[],0
1,A00-2023,\nThis paper presents a new approach to stati...,"{'T1': {'Annotation ID': 'T1', 'Entity': 'Gene...","{'R1': {'Relationship ID': 'R1', 'Type': 'COMP...","{'R1': {'Relationship ID': 'R1', 'Rel': 'ranki...",[ranking algorithm COMPARE lattice-based appro...,9,Give me a comma-separated list of relationship...,[approach FEATURE-OF statistical sentence gene...,8,[ranking algorithm COMPARE lattice-based appro...,9,[approach FEATURE-OF statistical sentence gene...,8,[],0
2,A88-1001,\n This paper describes a domain independent ...,"{'T1': {'Annotation ID': 'T1', 'Entity': 'Meth...","{'R1': {'Relationship ID': 'R1', 'Type': 'FEAT...","{'R1': {'Relationship ID': 'R1', 'Rel': 'feedb...","[feedback FEATURE-OF discourse, videodisc imag...",9,Give me a comma-separated list of relationship...,[natural language interface USED-FOR database ...,7,"[feedback FEATURE-OF discourse, videodisc imag...",8,[multimedia answers FEATURE-OF videodisc image...,6,[natural language interface USED-FOR database ...,1
3,A88-1003,"\n In this paper, we describe the pronominal...","{'T1': {'Annotation ID': 'T1', 'Entity': 'Meth...","{'R1': {'Relationship ID': 'R1', 'Type': 'PART...","{'R1': {'Relationship ID': 'R1', 'Rel': 'prono...",[pronominal anaphora resolution module PART-OF...,3,Give me a comma-separated list of relationship...,[pronominal anaphora resolution module PART-OF...,7,[module COREF pronominal anaphora resolution m...,2,"[anaphora resolution HYPONYM-OF theories, blac...",6,[pronominal anaphora resolution module PART-OF...,1
4,A92-1010,\nIn our current research into the design of ...,"{'T1': {'Annotation ID': 'T1', 'Entity': 'Task...","{'R1': {'Relationship ID': 'R1', 'Type': 'USED...","{'R1': {'Relationship ID': 'R1', 'Rel': 'displ...",[display of graphical information USED-FOR cog...,3,Give me a comma-separated list of relationship...,[natural language generation USED-FOR interact...,3,[display of graphical information USED-FOR cog...,2,[graphical information FEATURE-OF cognitively ...,2,[natural language generation USED-FOR interact...,1


### Export scierc_full_responses with additional analyses

In [110]:
reimport_scierc_results.to_excel("./benchmarking/results/additional_stats_scierc_rel_detection_with_ent_type_responses.xlsx")


## iterate over input annotations to get distribution of relation types

In [79]:
# Assuming 'reimport_scierc_results' is your DataFrame
simplified_populated_rels_list = [str(item) for item in reimport_scierc_results['simplified_populated_rels']]

In [25]:
# Define the target texts
target_texts = ['USED-FOR', 'FEATURE-OF', 'HYPONYM-OF', 'PART-OF', 'EVALUATE-FOR', 'COMPARE', 'CONJUNCTION', 'COREF']

# Initialize the dictionary
rel_summary = {text: 0 for text in target_texts}

# Count the occurrences of each target text
for row in simplified_populated_rels_list:
    phrases = row.split(', ')
    for text in target_texts:
        for phrase in phrases:
            if text in phrase:
                rel_summary[text] += 1

# Print the summary statistics
print(rel_summary)

# Print the sum of values
print("Sum of Values:", sum(rel_summary.values()))


{'USED-FOR': 2437, 'FEATURE-OF': 264, 'HYPONYM-OF': 477, 'PART-OF': 270, 'EVALUATE-FOR': 454, 'COMPARE': 234, 'CONJUNCTION': 583, 'COREF': 1675}
Sum of Values: 6394


## Explore 10 randomly selected samples where ChatGPT detected more rels than was annotated

In [27]:
# seedset = 3409578

# # Filter the DataFrame based on the condition
# responses_more_than_annotated = reimport_scierc_results[reimport_scierc_results['responses_rels_detected'] > reimport_scierc_results['rel_count']]
# responses_less_than_annotated = reimport_scierc_results[reimport_scierc_results['responses_rels_detected'] < reimport_scierc_results['rel_count']]


# # Randomly select 10 samples from the filtered DataFrame
# CGPT_detected_more = responses_more_than_annotated.sample(n=10, random_state=random.seed(seedset))
# CGPT_detected_less = responses_less_than_annotated.sample(n=10, random_state=random.seed(seedset))

# # Export the DataFrames to an Excel file with two sheets
# with pd.ExcelWriter('./benchmarking/results/qual_analysis_scierc.xlsx') as writer:
#     CGPT_detected_more.to_excel(writer, sheet_name='CGPT_detected_more')
#     CGPT_detected_less.to_excel(writer, sheet_name='CGPT_detected_less')